# Notes to Improve Workflow

In [127]:
# Setup matplotlib to plot inline (within the notebook)
%matplotlib inline
# Import the pyplot module of Matplotlib as plt
import matplotlib.pyplot as plt
# Import pandas under the abbreviation 'pd'
import pandas as pd
# Import NumPy under the abbreviation 'np'
import numpy as np

#Notes for efficienccy:
#shift + tab when cursor is in arguments to bring up its documentation
#ex: pd.DataFrame(Shift+tab here)
#recall: Json == Dictionary
#to know functions of pandas:
dir(pd)

['BooleanDtype',
 'Categorical',
 'CategoricalDtype',
 'CategoricalIndex',
 'DataFrame',
 'DateOffset',
 'DatetimeIndex',
 'DatetimeTZDtype',
 'ExcelFile',
 'ExcelWriter',
 'Flags',
 'Float32Dtype',
 'Float64Dtype',
 'Float64Index',
 'Grouper',
 'HDFStore',
 'Index',
 'IndexSlice',
 'Int16Dtype',
 'Int32Dtype',
 'Int64Dtype',
 'Int64Index',
 'Int8Dtype',
 'Interval',
 'IntervalDtype',
 'IntervalIndex',
 'MultiIndex',
 'NA',
 'NaT',
 'NamedAgg',
 'Period',
 'PeriodDtype',
 'PeriodIndex',
 'RangeIndex',
 'Series',
 'SparseDtype',
 'StringDtype',
 'Timedelta',
 'TimedeltaIndex',
 'Timestamp',
 'UInt16Dtype',
 'UInt32Dtype',
 'UInt64Dtype',
 'UInt64Index',
 'UInt8Dtype',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__docformat__',
 '__file__',
 '__getattr__',
 '__git_version__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_config',
 '_hashtable',
 '_is_numpy_dev',
 '_lib',
 '_libs',
 '_np_version_under1p18',
 '_testing',
 '_tslib',
 '_typing',
 

# Reading The Data

In [68]:
# using pandas and read_csv()
acquis = pd.read_csv("datasets/startups/acquisitions.csv")
additions = pd.read_csv("datasets/startups/additions.csv")
companies = pd.read_csv("datasets/startups/companies.csv")
invests = pd.read_csv("datasets/startups/investments.csv")
rounds = pd.read_csv("datasets/startups/rounds.csv")
dfAll = [acquis, additions, companies, invests, rounds]

In [69]:
#shapes (rows x cols) of each dataset
for ele in dfAll:
    print(ele.shape)

(18969, 18)
(2213, 4)
(66368, 14)
(168647, 18)
(114949, 12)


# Samples of the datasets

In [70]:
acquis.sample(5).transpose()

,12523,9479,4054,11051,9967
company_permalink,/organization/paydiant,/organization/lexicon-marketing,/organization/cope-today,/organization/myanimelist,/organization/mainstar-software
company_name,Paydiant,Lexicon Marketing,Cope Today,MyAnimeList,Mainstar Software
company_category_list,Mobile Commerce|Mobile Payments,Development Platforms|Language Learning|Video,NaN,Games,NaN
company_country_code,USA,NaN,USA,USA,USA
company_state_code,MA,NaN,NC,CA,WA
company_region,Boston,NaN,Raleigh,Ontario - Inland Empire,Seattle
company_city,Wellesley,NaN,Raleigh,Yorba Linda,Bellevue
acquirer_permalink,/organization/paypal,/organization/golden-gate-capital,/organization/mindcare-solutions,/organization/craveonline,/organization/rocket-software
acquirer_name,PayPal,Golden Gate Capital,MindCare Solutions,CraveOnline,Rocket Software
acquirer_category_list,E-Commerce|Finance|FinTech|Mobile Payments|P2P...,NaN,Health Services Industry,Curated Web|Mens Specific,Software


In [71]:
additions.sample(5).transpose()

,900,65,104,911,781
content,Profiles: Organizations,Acquisitions,Acquisitions,Profiles: Organizations,Profiles
month_str,2012-05,1996-08,1999-11,2013-04,2011-02
year_str,2012,1996,1999,2013,2011
value,2237,4,19,8020,4959


In [72]:
companies.sample(5).transpose()

,40127,39531,38191,21909,32096
permalink,/organization/nutzvieh24,/organization/node-management,/organization/napera-networks,/organization/funky-moves,/organization/letsventure
name,Nutzvieh24,Node Management,Napera Networks,Funky Moves,LetsVenture
homepage_url,http://www.vieh24.de,NaN,http://www.napera.com,http://www.funky-moves.co.uk,http://letsventure.com
category_list,Finance,Software,Networking|Network Security|SaaS|Security,Games,Internet
funding_total_usd,-,960000,6600000,520094,650000
status,closed,operating,acquired,operating,operating
country_code,DEU,NaN,USA,GBR,SGP
state_code,2,NaN,WA,W9,NaN
region,DEU - Other,NaN,Seattle,Livingston,Singapore
city,Miesbach,NaN,Bellevue,Livingston,Singapore


In [73]:
invests.sample(5).transpose()

,159548,161148,23504,79027,61226
company_permalink,/organization/vserv,/organization/weddington-way,/organization/bubbles-and-beyond,/organization/kwanji,/organization/graze
company_name,Vserv,Weddington Way,Bubbles and Beyond,Kwanji,Graze
company_category_list,Emerging Markets|Mobile Advertising|Mobile Com...,Consumer Internet|E-Commerce|Events|Fashion|So...,Biotechnology,Finance|Shipping|Small and Medium Businesses|S...,E-Commerce
company_country_code,IND,USA,DEU,GBR,USA
company_state_code,16,CA,13,H9,NY
company_region,Mumbai,SF Bay Area,Leipzig,London,New York City
company_city,Mumbai,San Francisco,Leipzig,London,New York
investor_permalink,/organization/epiphany-ventures,/organization/felicis-ventures,/organization/high-tech-gruenderfonds,/organization/midsummer-capital,/organization/octopus-ventures
investor_name,Epiphany Ventures,Felicis Ventures,High-Tech Gruenderfonds,Midsummer Capital,Octopus Ventures
investor_country_code,IND,USA,DEU,USA,GBR


In [74]:
rounds.sample(5).transpose()

,31892,4865,1490,65747,76552
company_permalink,/organization/enertiv,/organization/alterg,/organization/accera,/organization/nanomr,/organization/plug-apps
company_name,Enertiv,AlterG,Accera,nanoMR,Plug Apps
company_category_list,Analytics|Energy Efficiency,Health Care|Medical Devices,Biotechnology,Biotechnology,Apps|Databases|Internet|Personalization
company_country_code,NaN,USA,USA,USA,NaN
company_state_code,NaN,CA,CO,NM,NaN
company_region,NaN,SF Bay Area,Denver,Albuquerque,NaN
company_city,NaN,Fremont,Broomfield,Albuquerque,NaN
funding_round_permalink,/funding-round/79627fdd6199f0567dc37393975d95b7,/funding-round/527a8af29fabe1c6d0a49422dd74aeac,/funding-round/e6658050ba8d6d2e23060ca6e7ae8d92,/funding-round/8e794b80ff062c4eb923adf09ff713df,/funding-round/b08df1c0884fe8e734336d0c0d4945f6
funding_round_type,seed,venture,venture,debt_financing,seed
funding_round_code,NaN,NaN,C,NaN,NaN


In [75]:
#getting all (unique) columns of the datasets
uniqueCols = set() #or: instead of "{}", "set()"
for ele in dfAll:
    cols = ele.columns.tolist() #".tolist()" to make it a list (side note: ".values" to make it a numpy.ndarray)
    
    uniqueCols.update(cols) #use ".add()" for elements, ".update()" for lists 

In [76]:
#sorting the columns alphabetically
tempList = list(uniqueCols)
tempList.sort()
uniqueCols = tempList
uniqueCols

['acquired_at',
 'acquired_month',
 'acquirer_category_list',
 'acquirer_city',
 'acquirer_country_code',
 'acquirer_name',
 'acquirer_permalink',
 'acquirer_region',
 'acquirer_state_code',
 'category_list',
 'city',
 'company_category_list',
 'company_city',
 'company_country_code',
 'company_name',
 'company_permalink',
 'company_region',
 'company_state_code',
 'content',
 'country_code',
 'first_funding_at',
 'founded_at',
 'funded_at',
 'funding_round_code',
 'funding_round_permalink',
 'funding_round_type',
 'funding_rounds',
 'funding_total_usd',
 'homepage_url',
 'investor_city',
 'investor_country_code',
 'investor_name',
 'investor_permalink',
 'investor_region',
 'investor_state_code',
 'last_funding_at',
 'month_str',
 'name',
 'permalink',
 'price_amount',
 'price_currency_code',
 'raised_amount_usd',
 'region',
 'state_code',
 'status',
 'value',
 'year_str']

# Seeing the columns in the datasets

In [77]:
def datasetsCols(dfList, datasetNames, pad=""):
    """
    Takes dataframes and returns a dataframe (df) with each column having a df's
    column names. The "pad" is to make sure all lists of columns have 
    the same length padded out with "pad"
    Example
    pad = "XX", datasetNames = ["cars", "planes"]
    dfList = [carsDF, planesDF]
    returned data frame:
            cars            planes
    0       numOfWheels     numOfWings
    1       manufacturer    manufacturer
    2       make            XX
    """
    cols = []
    maxArrayLen = 0
    for i, df in enumerate(dfList):
        cols.append(df.columns.tolist())
        maxArrayLen = max(maxArrayLen, len(cols[i]))
    
    dictCsvs = {}
    for i, df in enumerate(dfList):
        cols[i] += [pad] * (maxArrayLen - len(cols[i])) #padding the lists to make them have equal lengths 
        dictCsvs.update({datasetNames[i] : cols[i]})

    return pd.DataFrame(dictCsvs)

In [78]:
lstCsvNames = ["acquisitions", "additions", "companies", "investments", "rounds"]

dfAllCols = datasetsCols(dfAll, lstCsvNames)
dfAllCols

,acquisitions,additions,companies,investments,rounds
0,company_permalink,content,permalink,company_permalink,company_permalink
1,company_name,month_str,name,company_name,company_name
2,company_category_list,year_str,homepage_url,company_category_list,company_category_list
3,company_country_code,value,category_list,company_country_code,company_country_code
4,company_state_code,,funding_total_usd,company_state_code,company_state_code
5,company_region,,status,company_region,company_region
6,company_city,,country_code,company_city,company_city
7,acquirer_permalink,,state_code,investor_permalink,funding_round_permalink
8,acquirer_name,,region,investor_name,funding_round_type
9,acquirer_category_list,,city,investor_country_code,funding_round_code


# Data Cleaning

First, lets change the columns in the `companies` dataset <br>
to be the similar to all other datasets:

In [79]:
companies.rename(columns = {'permalink' : 'company_permalink', 
                            'name' : 'company_name',
                            'category_list' : 'company_category_list',
                            'country_code' : 'company_country_code',
                            'state_code' : 'company_state_code',
                            'region' : 'company_region',
                            'city' : 'company_city'}, inplace = True)

In [80]:
dfAll[2] = companies
dfAllCols = datasetsCols(dfAll, lstCsvNames)
dfAllCols

,acquisitions,additions,companies,investments,rounds
0,company_permalink,content,company_permalink,company_permalink,company_permalink
1,company_name,month_str,company_name,company_name,company_name
2,company_category_list,year_str,homepage_url,company_category_list,company_category_list
3,company_country_code,value,company_category_list,company_country_code,company_country_code
4,company_state_code,,funding_total_usd,company_state_code,company_state_code
5,company_region,,status,company_region,company_region
6,company_city,,company_country_code,company_city,company_city
7,acquirer_permalink,,company_state_code,investor_permalink,funding_round_permalink
8,acquirer_name,,company_region,investor_name,funding_round_type
9,acquirer_category_list,,company_city,investor_country_code,funding_round_code


Let's start by analyzing `additions` dataset, as it has unusual format of columns

## Removing "additions" Dataset

In [81]:
additions.sample(5)

,content,month_str,year_str,value
1390,Rounds: convertible_note,2014-01,2014,42
898,Profiles: Organizations,2012-03,2012,2069
275,Acquisitions,2014-02,2014,192
1764,Rounds: private_equity,2009-02,2009,2
1821,Rounds: private_equity,2013-12,2013,37


In [82]:
additions["content"].unique()

array(['Acquisitions', 'IPOs', 'Profiles', 'Profiles: Organizations',
       'Profiles: People', 'Profiles: Products', 'Rounds',
       'Rounds: angel', 'Rounds: convertible_note',
       'Rounds: debt_financing', 'Rounds: equity_crowdfunding',
       'Rounds: grant', 'Rounds: non_equity_assistance',
       'Rounds: post_ipo_debt', 'Rounds: post_ipo_equity',
       'Rounds: private_equity', 'Rounds: product_crowdfunding',
       'Rounds: secondary_market', 'Rounds: seed', 'Rounds: undisclosed',
       'Rounds: venture'], dtype=object)

In [83]:
dfAllCols

,acquisitions,additions,companies,investments,rounds
0,company_permalink,content,company_permalink,company_permalink,company_permalink
1,company_name,month_str,company_name,company_name,company_name
2,company_category_list,year_str,homepage_url,company_category_list,company_category_list
3,company_country_code,value,company_category_list,company_country_code,company_country_code
4,company_state_code,,funding_total_usd,company_state_code,company_state_code
5,company_region,,status,company_region,company_region
6,company_city,,company_country_code,company_city,company_city
7,acquirer_permalink,,company_state_code,investor_permalink,funding_round_permalink
8,acquirer_name,,company_region,investor_name,funding_round_type
9,acquirer_category_list,,company_city,investor_country_code,funding_round_code


In [84]:
additions.shape

(2213, 4)

In [85]:
companies.shape, rounds.shape

((66368, 14), (114949, 12))

So apparently there are no columns that could be used to join with the other datasets. <br>
Furthermore, the "value" column is too vague to be useful. <br>
Therefore, the "additions" dataset will be discarded.

In [86]:
dfAll.pop(1)
lstCsvNames.pop(1)
len(dfAll), lstCsvNames

(4, ['acquisitions', 'companies', 'investments', 'rounds'])

# Data Cleaning (Cont.)

## Checking All Missing Values

In [87]:
def datasetsNulls(dfList, datasetNames, nullCol="nulls_", pad=""):
    """
    Use this when you want to display a column of column names,
    then a column of the null values, and repeat that for each dataset.
    Returns a dataframe
    """
    cols = []
    nulls = []
    maxArrayLen = 0
    for i, df in enumerate(dfList):
        cols.append(df.columns.tolist())
        nulls.append(df.isnull().sum().tolist())
        maxArrayLen = max(maxArrayLen, len(cols[i]))
    
    dictCsvs = {}
    for i, df in enumerate(dfList):
        cols[i] += [pad] * (maxArrayLen - len(cols[i])) #padding the lists to make them have equal lengths 
        nulls[i] += [None] *  (maxArrayLen - len(nulls[i]))
        dictCsvs[datasetNames[i]] = cols[i]
        dictCsvs[nullCol + str(i+1)] = nulls[i]
    
    return pd.DataFrame(dictCsvs)

In [88]:
dfAllNulls = datasetsNulls(dfAll, lstCsvNames)
dfAllNulls

,acquisitions,nulls_1,companies,nulls_2,investments,nulls_3,rounds,nulls_4
0,company_permalink,0,company_permalink,0.0,company_permalink,0,company_permalink,0.0
1,company_name,1,company_name,1.0,company_name,1,company_name,1.0
2,company_category_list,4984,homepage_url,5058.0,company_category_list,3632,company_category_list,3410.0
3,company_country_code,4669,company_category_list,3148.0,company_country_code,10521,company_country_code,8678.0
4,company_state_code,4949,funding_total_usd,0.0,company_state_code,13431,company_state_code,10946.0
5,company_region,4911,status,0.0,company_region,12485,company_region,10167.0
6,company_city,4911,company_country_code,6958.0,company_city,12483,company_city,10164.0
7,acquirer_permalink,1,company_state_code,8547.0,investor_permalink,12,funding_round_permalink,0.0
8,acquirer_name,1,company_region,8030.0,investor_name,12,funding_round_type,0.0
9,acquirer_category_list,2389,company_city,8028.0,investor_country_code,30939,funding_round_code,83809.0


Alternatively, we can combine all columns to be the dataframe's index:

In [89]:
#Alternatively:
def datasetsNullsSameIndex(datasetsNames, *args):
    """
    Use this when multiple datasets have a lot of columns
    with the same names.
    Returns a dataframe with the index value being the 
    columns of all datasets
    """
    dic = {}
    for i, df in enumerate(args):
        dic[datasetsNames[i]] = df.isnull().sum()
    return pd.DataFrame(dic)

In [90]:
datasetsNullsSameIndex(lstCsvNames, *dfAll)

,acquisitions,companies,investments,rounds
acquired_at,13.0,NaN,NaN,NaN
acquired_month,15.0,NaN,NaN,NaN
acquirer_category_list,2389.0,NaN,NaN,NaN
acquirer_city,1360.0,NaN,NaN,NaN
acquirer_country_code,1171.0,NaN,NaN,NaN
acquirer_name,1.0,NaN,NaN,NaN
acquirer_permalink,1.0,NaN,NaN,NaN
acquirer_region,1360.0,NaN,NaN,NaN
acquirer_state_code,1436.0,NaN,NaN,NaN
company_category_list,4984.0,3148.0,3632.0,3410.0


### `dropna()` for columns with few missing values

Notice that there is one row that doesn't contain a company name, <br>
so let's validate that it is the same company across all datasets, <br>
so we can remove it:

In [91]:
datasetsCols(dfAll, lstCsvNames)

,acquisitions,companies,investments,rounds
0,company_permalink,company_permalink,company_permalink,company_permalink
1,company_name,company_name,company_name,company_name
2,company_category_list,homepage_url,company_category_list,company_category_list
3,company_country_code,company_category_list,company_country_code,company_country_code
4,company_state_code,funding_total_usd,company_state_code,company_state_code
5,company_region,status,company_region,company_region
6,company_city,company_country_code,company_city,company_city
7,acquirer_permalink,company_state_code,investor_permalink,funding_round_permalink
8,acquirer_name,company_region,investor_name,funding_round_type
9,acquirer_category_list,company_city,investor_country_code,funding_round_code


In [92]:
#company_permalink of each row that doesn't have a company name:
for df in dfAll:
    print(df[df['company_name'].isnull()]['company_permalink'].values)

['mendatio"']
['/organization/tell-it-in']
['/organization/tell-it-in']
['/organization/tell-it-in']


So apparently, the `acquisitions` dataset has a different permalink that has no company name, <br>
So we could remove these two companies, as the total number of companies are big enough to get the insight that we want: 

In [93]:
for df in dfAll:
    df.dropna(subset=['company_name'], inplace=True)
#Check:
for df in dfAll:
    print(df[df['company_name'].isnull()]['company_permalink'].values)

[]
[]
[]
[]


`dfAllNulls` dataframe also shows that there are missing values for `acquirer_name` and `investor_name` <br>
which can't be imputed, so we'll also remove them:

In [94]:
# Note: same logic applies for the permalinks
acquis.dropna(subset=['acquirer_permalink', 'acquirer_name'], inplace=True)
invests.dropna(subset=['investor_permalink', 'investor_name'], inplace=True)

Also, we'll remove the missing values for columns that have less than 100 `NaN`, <br>
as they're small in comparison to their respective datasets:

In [95]:
for df in dfAll:
    nulls = df.isnull().sum()
    cols = nulls[(nulls <= 100) & (nulls != 0)].index.tolist()
    df.dropna(subset=cols, inplace=True)
dfAllNulls = datasetsNulls(dfAll, lstCsvNames)
dfAllNulls

,acquisitions,nulls_1,companies,nulls_2,investments,nulls_3,rounds,nulls_4
0,company_permalink,0,company_permalink,0.0,company_permalink,0,company_permalink,0.0
1,company_name,0,company_name,0.0,company_name,0,company_name,0.0
2,company_category_list,4977,homepage_url,5058.0,company_category_list,3632,company_category_list,3410.0
3,company_country_code,4661,company_category_list,3147.0,company_country_code,10521,company_country_code,8678.0
4,company_state_code,4940,funding_total_usd,0.0,company_state_code,13431,company_state_code,10946.0
5,company_region,4902,status,0.0,company_region,12485,company_region,10167.0
6,company_city,4902,company_country_code,6956.0,company_city,12483,company_city,10164.0
7,acquirer_permalink,0,company_state_code,8545.0,investor_permalink,0,funding_round_permalink,0.0
8,acquirer_name,0,company_region,8028.0,investor_name,0,funding_round_type,0.0
9,acquirer_category_list,2387,company_city,8026.0,investor_country_code,30927,funding_round_code,83808.0


## Converting & Imputing Dates

[SAY WHAT YOU WILL DO HERE FARAH:]

In [96]:
#Farah: plz impute (not drop) the columns' missing values related to dates in each dataset which are:
#founded_at.. and that's it :] 
# Suggestion: convert the dates of all the columns that have date values from "object" to datetime
#       Helpful link: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#converting-to-timestamps
# Then, make the founded_at a couple of days before first_funding_at

#Sincerely No. -Farah :)

In [97]:
#farah stop point here

## Cleaning & Imputing Geographical Locations

### Removing `company_state_code` and `company_region`

We don't need `company_state_code` in our analysis, <br>
As country code and city are sufficient to know <br>
an approximation of the geographical location of the company. <br>
Therefore, let's remove it:

In [98]:
companies[['company_country_code', 'company_state_code', 'company_city']].head(10)

,company_country_code,company_state_code,company_city
0,IND,16,Mumbai
1,USA,DE,Delaware City
2,NaN,NaN,NaN
3,CHN,22,Beijing
4,USA,IL,Champaign
5,HKG,NaN,Hong Kong
6,CAN,BC,Vancouver
7,USA,CA,Mountain View
8,USA,CA,San Francisco
9,CHL,12,Las Condes


In [99]:
companies.drop('company_state_code', axis=1, inplace=True)

`company_region` is also redundent, as it is usually the same as `company_city`.<br>
Let's validate this:

In [118]:
companies[companies.company_region == companies.company_city][['company_region', 'company_city']]

,company_region,company_city
0,Mumbai,Mumbai
3,Beijing,Beijing
5,Hong Kong,Hong Kong
6,Vancouver,Vancouver
11,Orlando,Orlando
...,...,...
66354,Bath,Bath
66360,Milwaukee,Milwaukee
66362,London,London
66363,Beijing,Beijing


Let's see the mean of similarity between the strings using builtin library `difflib.SequenceMatcher`:

In [123]:
from difflib import SequenceMatcher
similarities = []
rgs = companies.company_region.values.tolist()
cts = companies.company_city.values.tolist()
for i in range(min(len(rgs), len(cts))):
    if not (rgs[i] != rgs[i] or cts[i] != cts[i]): # If either of the elements is NaN, then don't find the similarity
        similarities.append(SequenceMatcher(None, rgs[i], cts[i]).ratio()) # Gets the similarity between each two strings
avgSim = sum(similarities) / len(similarities)
avgSim

0.6175093624179544

However, `SequenceMatcher` only finds the longest contiguous matching subsequence, <br>
and all other characters will drastically decrease the similarity, which isn't very accurate, <br>
Therefore, let's try to see how many times a string is a substring of another:

In [124]:
subsCount = 0
for i in range(min(len(rgs), len(cts))):
    if not (rgs[i] != rgs[i] or cts[i] != cts[i]):
        subsCount += (rgs[i] in cts[i] or cts[i] in rgs[i])
subsCount

29884

So to summarize: almost half of the companies have the same data for `region` and `city` (25360) <br>
There are around 5000 companies that have the same semantic meaning between the data <br>
(e.g: 'New York City' is the same as 'NYC, New York City') <br>
Therefore, `company_region` can be removed without losing any possible future insights:

In [125]:
companies.drop('company_region', axis=1, inplace=True)

### Imputing `company_country_code` and `company_city`

A way to impute country codes is to do the following:
1. check the company's permalink
2. find HTML that contains 

In [100]:
companies[companies.company_country_code.isnull() == False].sample(10)

,company_permalink,company_name,homepage_url,company_category_list,funding_total_usd,status,company_country_code,company_region,company_city,funding_rounds,founded_at,first_funding_at,last_funding_at
59235,/organization/trending-taste,Trending Taste,http://trendingtaste.com/,Entertainment|Games,-,operating,USA,SF Bay Area,San Jose,1,6/22/2013,7/2/2014,7/2/2014
38588,/organization/neozone,Neozone,http://www.neozone.com,Software,809000,operating,DNK,Copenhagen,Copenhagen,1,NaN,1/17/2006,1/17/2006
60491,/organization/united-digital-group,United Digital Group,http://www.udg.de/,Advertising,-,operating,DEU,Hamburg,Hamburg,1,NaN,6/1/2011,6/1/2011
34370,/organization/marquee-productions-inc,MPI,http://www.marqueeworld.com,Music,-,operating,USA,Los Angeles,Malibu,1,10/4/2010,10/10/2010,10/10/2010
60156,/organization/uber-com,Uber.com,http://www.dmwmedia.com/news/2008/09/29/social...,Communications Infrastructure|Media|Social Net...,7600000,closed,USA,Los Angeles,Los Angeles,1,NaN,5/26/2008,5/26/2008
2138,/organization/aireon,Aireon,http://www.aireon.com,Public Transportation,50000000,operating,USA,"Washington, D.C.",Mclean,1,NaN,2/18/2014,2/18/2014
53134,/organization/solar-universe,Solar Universe,http://solaruniverse.com,Clean Energy|Clean Technology|Franchises|Renew...,11000000,operating,USA,SF Bay Area,Livermore,2,1/1/2007,1/25/2011,4/20/2015
39344,/organization/nifti,Nifti,http://nifti.com,Curated Web,800000,operating,USA,Boston,Boston,1,1/1/2012,7/18/2013,7/18/2013
28210,/organization/integrys-assetpoint,Integrys AssetPoint,http://www.assetpoint.com,Business Services|Productivity Software|Software,5500000,operating,USA,Greenville - Spartanburg,Greenville,1,1/1/2002,3/6/2008,3/6/2008
23730,/organization/gozaround-inc,GozAround Inc.,http://www.gozaround.com,Non Profit|Social Commerce|Social Media,46000,operating,CAN,Edmonton,Edmonton,1,1/15/2014,1/15/2014,1/15/2014


In [101]:
companies[companies.company_country_code.isnull() == False].sample(5)

,company_permalink,company_name,homepage_url,company_category_list,funding_total_usd,status,company_country_code,company_region,company_city,funding_rounds,founded_at,first_funding_at,last_funding_at
50151,/organization/seattle-coffee-company,Seattle Coffee Company,NaN,Coffee|E-Commerce|Food Processing,-,operating,USA,New York City,Astoria,1,2/1/2014,2/10/2013,2/10/2013
12681,/organization/compassinc,Compass,http://www.compass.com,Curated Web|Local|Mobile|Real Estate,123000000,operating,USA,New York City,New York,4,10/1/2012,12/16/2012,9/15/2015
36208,/organization/mission-research,Mission Research,http://www.circledog.com,Nanotechnology,1500000,acquired,USA,Harrisburg,Lancaster,2,2/15/2002,11/30/2006,2/25/2008
10551,/organization/cetek-corporation,Cetek Corporation,http://www.cetek.com/,Bio-Pharm,20000000,operating,USA,Boston,Marlborough,2,1/1/1996,10/12/2001,4/2/2003
12807,/organization/concloud,ConCloud,http://www.concloud.com,Android|Apps|Cloud Computing|Contact Managemen...,100000,operating,DEU,Frankfurt,Frankfurt,1,2/28/2014,2/28/2014,2/28/2014


In [102]:
companies.company_country_code.unique()

array(['IND', 'USA', nan, 'CHN', 'HKG', 'CAN', 'CHL', 'GBR', 'FRA', 'AUS',
       'DNK', 'ROM', 'AUT', 'KOR', 'NLD', 'SWE', 'JPN', 'RUS', 'SGP',
       'NOR', 'COL', 'ESP', 'BEL', 'IRL', 'ITA', 'ISR', 'NZL', 'CZE',
       'DEU', 'CHE', 'BRA', 'BGR', 'SVN', 'JOR', 'HUN', 'BWA', 'NGA',
       'FIN', 'TUR', 'IDN', 'ARE', 'CYP', 'POL', 'CRI', 'PRT', 'ARG',
       'TWN', 'KHM', 'THA', 'SVK', 'UKR', 'LTU', 'ISL', 'ZAF', 'MEX',
       'VEN', 'URY', 'MUS', 'KEN', 'PHL', 'MNE', 'VNM', 'GHA', 'PSE',
       'MYS', 'PER', 'EGY', 'PAN', 'LVA', 'GGY', 'ALB', 'UGA', 'HRV',
       'EST', 'LBN', 'GRC', 'PAK', 'NPL', 'LUX', 'SAU', 'IRN', 'GTM',
       'BAH', 'ARM', 'BGD', 'MDA', 'SRB', 'TUN', 'ZWE', 'BRB', 'NIC',
       'TAN', 'PRI', 'TTO', 'BHR', 'CMR', 'BLR', 'CYM', 'BRN', 'ECU',
       'SLV', 'MLT', 'ZMB', 'MKD', 'GIB', 'TGO', 'LAO', 'BMU', 'HND',
       'MCO', 'MMR', 'KAZ', 'QAT', 'DOM', 'MAR', 'LIE', 'GEO', 'MOZ',
       'DZA', 'JAM', 'KNA', 'AZE', 'SEN', 'RWA', 'DMA', 'UZB', 'BLZ',
       'OMN', '

## Checking All `dtypes`

In [103]:
def datasetsDtypes(dfList, datasetNames, dtypesCol="dtypes_", pad=""):
    cols = []
    nulls = []
    maxArrayLen = 0
    for i, df in enumerate(dfList):
        cols.append(df.columns.tolist())
        nulls.append(df.dtypes.tolist())
        maxArrayLen = max(maxArrayLen, len(cols[i]))
    
    dictCsvs = {}
    for i, df in enumerate(dfList):
        cols[i] += [pad] * (maxArrayLen - len(cols[i])) #padding the lists to make them have equal lengths 
        nulls[i] += [None] *  (maxArrayLen - len(nulls[i]))
        dictCsvs[datasetNames[i]] = cols[i]
        dictCsvs[dtypesCol + str(i+1)] = nulls[i]
    
    return pd.DataFrame(dictCsvs)

In [104]:
datasetsDtypes(dfAll, lstCsvNames)

,acquisitions,dtypes_1,companies,dtypes_2,investments,dtypes_3,rounds,dtypes_4
0,company_permalink,object,company_permalink,object,company_permalink,object,company_permalink,object
1,company_name,object,company_name,object,company_name,object,company_name,object
2,company_category_list,object,homepage_url,object,company_category_list,object,company_category_list,object
3,company_country_code,object,company_category_list,object,company_country_code,object,company_country_code,object
4,company_state_code,object,funding_total_usd,object,company_state_code,object,company_state_code,object
5,company_region,object,status,object,company_region,object,company_region,object
6,company_city,object,company_country_code,object,company_city,object,company_city,object
7,acquirer_permalink,object,company_region,object,investor_permalink,object,funding_round_permalink,object
8,acquirer_name,object,company_city,object,investor_name,object,funding_round_type,object
9,acquirer_category_list,object,funding_rounds,int64,investor_country_code,object,funding_round_code,object


In [3]:
#AISHA: see what you can do with the dtypes above (except dates, that's on Farah) ^^

testing something here


In [1]:
print("Aisha is happy ^^")

Aisha is happy ^^


In [106]:
#Aisha stop

## `rounds` Dataset

In [107]:
dfAllCols.iloc[:, -2:]

,investments,rounds
0,company_permalink,company_permalink
1,company_name,company_name
2,company_category_list,company_category_list
3,company_country_code,company_country_code
4,company_state_code,company_state_code
5,company_region,company_region
6,company_city,company_city
7,investor_permalink,funding_round_permalink
8,investor_name,funding_round_type
9,investor_country_code,funding_round_code


In [108]:
invests.shape, rounds.shape

((168634, 18), (114948, 12))

In [109]:
invests.head()

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/0-6-com,0-6.com,Curated Web,CHN,22,Beijing,Beijing,/organization/dt-capital-partners,DT Capital Partners,CHN,23,Shanghai,Shanghai,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,3/19/2008,2000000.0
1,/organization/004-technologies,004 Technologies,Software,USA,IL,"Springfield, Illinois",Champaign,/organization/venturecapital-de,VCDE Venture Partners,DEU,5,Frankfurt,Frankfurt,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,7/24/2014,NaN
2,/organization/01games-technology,01Games Technology,Games,HKG,NaN,Hong Kong,Hong Kong,/organization/cyberport-hong-kong,Cyberport Hong Kong,HKG,NaN,Hong Kong,Hong Kong,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,NaN,7/1/2014,41250.0
3,/organization/0xdata,H2O.ai,Analytics,USA,CA,SF Bay Area,Mountain View,/organization/capital-one,Capital One,USA,VA,"Washington, D.C.",Mclean,/funding-round/3bb2ee4a2d89251a10aaa735b1180e44,venture,B,11/9/2015,20000000.0
4,/organization/0xdata,H2O.ai,Analytics,USA,CA,SF Bay Area,Mountain View,/organization/nexus-venture-partners,Nexus Venture Partners,USA,CA,SF Bay Area,Menlo Park,/funding-round/383a9bd2c04f7038bb543ccef5ba3eae,seed,NaN,5/22/2013,3000000.0


In [110]:
rounds.head()

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,#fame,Media,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,1/5/2015,10000000.0
1,/organization/-qounter,:Qounter,Application Platforms|Real Time|Social Network...,USA,DE,DE - Other,Delaware City,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,10/14/2014,NaN
2,/organization/-qounter,:Qounter,Application Platforms|Real Time|Social Network...,USA,DE,DE - Other,Delaware City,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,3/1/2014,700000.0
3,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",Apps|Games|Mobile,NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,1/30/2014,3406878.0
4,/organization/0-6-com,0-6.com,Curated Web,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,3/19/2008,2000000.0


In [111]:
len(invests["company_name"].unique()), len(rounds["company_name"].unique())

(44577, 66102)

Since there are companies in `rounds` dataset that are not in `investments` dataset, <br>
therefore we should keep `rounds` dataset for further analysis.

## `companies` Dataset

In [112]:
companies.sample(5).T

,64028,54848,36853,52160,65008
company_permalink,/organization/windward-naval,/organization/storefront-net,/organization/money-dashboard,/organization/skuserve,/organization/yakarouler
company_name,Windward,StoreFront.net,Money Dashboard,SkuServe,Yakarouler
homepage_url,http://www.windward.eu/,http://www.storefront.net,http://www.moneydashboard.com,http://www.skuserve.com,http://www.yakarouler.com/
company_category_list,Big Data Analytics,E-Commerce,Curated Web|Personal Finance,NaN,Automotive|Discounts|Marketplaces
funding_total_usd,15800000,6000000,7791178,-,3925845
status,operating,acquired,operating,operating,operating
company_country_code,ISR,USA,GBR,USA,FRA
company_region,Tel Aviv,Kansas City,Edinburgh,Cleveland,FRA - Other
company_city,Tel Aviv,Olathe,Edinburgh,Cleveland,Thiais
funding_rounds,2,1,2,1,1


In [113]:
companies.isnull().sum()

company_permalink            0
company_name                 0
homepage_url              5058
company_category_list     3147
funding_total_usd            0
status                       0
company_country_code      6956
company_region            8028
company_city              8026
funding_rounds               0
founded_at               15218
first_funding_at             0
last_funding_at              0
dtype: int64

we will need to remove the none values of the following columns <br>
based on the question that we are trying to gain insight for: <br>
1.	Can the factors that affect a start-up’s growth be determined?
    -	For this: We are initially interested in <br> `funding_total_usd`, `status`, `funding_rounds`, `first_funding_at`, and `last_funding_at`
    -   So we could remove the 24 null values of `first_funding_at` since they are so small <br>
        compared to the rest of the dataset
2.  Do taxes play an important factor in deciding a start-up’s success?

